# Reproducing codebase as Python API

The notebook reproduces the trained-BART model as the Python API not as the CLI. 

In [1]:
import os
import sys
import argparse
import logging
import re
import typing as ty

from tqdm import tqdm
from warnings import warn
from torch.multiprocessing import Pool, set_start_method
set_start_method('spawn', force=True)
from functools import partial
import more_itertools as mit

import torch
import fairseq
from fairseq.models.bart import BARTHubInterface
from fairseq.models.bart import BARTModel

import nvgpu

from pathlib import Path

/home/kmitsuzawa/.local/miniconda3/envs/p39-Dreyer-2023/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import logzero

from datetime import datetime
_datetime_exec = datetime.now()

logzero.logfile(f"logs/{_datetime_exec.isoformat()}.log")

logger = logzero.logger

In [3]:
def load_model(task: Path, model_path: Path) -> BARTHubInterface:
    """
    Args:
        task: a path to the directory of the model.
        model_path: a path to 'model.pt' file.
    """
    assert task.exists()
    assert model_path.exists()

    logger.info(f"Loading model {model_path}")
    model_dirname, model_fname = os.path.split(model_path.as_posix())
    bart = BARTModel.from_pretrained(
        model_dirname,
        checkpoint_file=model_fname,
        data_name_or_path=task.as_posix()
    )
    logger.info(f"Loading done.")
    return bart


In [4]:
# path to input
PATH_TEXT_FILE_INPUT = Path("/workdir/kmitsuzawa/Project/neurips-2025/ConstraintsFact-Dreyer-2023/abstractive-factual-tradeoff/tests/testresources/xsum/test_source.txt")
assert PATH_TEXT_FILE_INPUT.exists()

seq_text_input = PATH_TEXT_FILE_INPUT.open().readlines()
assert len(seq_text_input) > 0

In [ ]:
# with xsum model
# PATH_MODEL_FILE = Path('/workdir/kmitsuzawa/Project/neurips-2025/ConstraintsFact-Dreyer-2023/abstractive-factual-tradeoff/tests/testresources/models/bart.large.xsum')
# with cnn model
PATH_MODEL_FILE = Path('/workdir/kmitsuzawa/Project/neurips-2025/ConstraintsFact-Dreyer-2023/abstractive-factual-tradeoff/tests/testresources/models/bart.large.cnn')

bart_model = load_model(PATH_MODEL_FILE, PATH_MODEL_FILE / 'model.pt')

[I 250709 10:50:41 2610531437:10] Loading model /workdir/kmitsuzawa/Project/neurips-2025/ConstraintsFact-Dreyer-2023/abstractive-factual-tradeoff/tests/testresources/models/bart.large.cnn/model.pt


In [ ]:
type(bart_model)

fairseq.models.bart.hub_interface.BARTHubInterface

In [ ]:
logger.info(str(bart_model))

[I 250709 10:47:28 3849268264:1] BARTHubInterface(
      (model): BARTModel(
        (encoder): TransformerEncoder(
          (embed_tokens): Embedding(50264, 1024, padding_idx=1)
          (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
          (layers): ModuleList(
            (0): TransformerEncoderLayer(
              (self_attn): MultiheadAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
              )
              (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
              (fc1): Linear(in_features=1024, out_features=4096, bias=True)
              (fc2): Linear(in_features=4096, out_features=1024, bias=True)
              (final_layer_no

In [ ]:
if torch.cuda.is_available():
    device_obj = torch.device('cuda:0')
else:
    device_obj = torch.device('cpu')
# end if

bart_model = bart_model.to(device_obj)

In [ ]:
def bart_sample(bart: BARTHubInterface,
                batch: ty.List[str],
                extractive_penalty_fct: str,
                beam: int = 4,
                lenpen: float = 2.0,  # length penalty
                min_len: int = 55,
                max_len_a: int = 0,
                max_len_b: int = 140,
                no_repeat_ngram_size: int = 3):
    with torch.no_grad():
        return bart.sample(batch, beam=beam, lenpen=lenpen,
                            min_len=min_len, max_len_a=max_len_a, max_len_b=max_len_b,
                            no_repeat_ngram_size=no_repeat_ngram_size,
                            extractive_penalty_fct=extractive_penalty_fct)
# end def


res = bart_sample(
    bart=bart_model,
    batch=seq_text_input,
    extractive_penalty_fct='log_exp(2,2.402244)'
)

TypeError: 'torch.device' object is not callable

In [ ]:
def bart_sample_stochastic(bart: BARTHubInterface,
                           batch: ty.List[str],
                           temperature: float,
                           extractive_penalty_fct: str,
                           random_seed_value: int,                           
                           beam: int = 4,
                           lenpen: float = 2.0,  # length penalty
                           min_len: int = 55,
                           max_len_a: int = 0,
                           max_len_b: int = 140,
                           no_repeat_ngram_size: int = 3):
    dict_parameters = dict(
        beam=beam,
        lenpen=lenpen,
        sampling=True,
        temperature=temperature,
        min_len=min_len, 
        max_len_a=max_len_a, 
        max_len_b=max_len_b,
        no_repeat_ngram_size=no_repeat_ngram_size,
        extractive_penalty_fct=extractive_penalty_fct)

    with (torch.random.fork_rng(), torch.no_grad()):
        torch.manual_seed(random_seed_value)
        torch.cuda.manual_seed_all(random_seed_value)  # if you are using multi-GPU.        
        res = bart.sample(batch, **dict_parameters)

    return res
# end def


---

In [ ]:
# # case Xsum constraints dataset
# import json

# PATH_CONSTRAINS_XSUM = Path("/workdir/kmitsuzawa/Project/neurips-2025/ConstraintsFact-Dreyer-2023/abstractive-factual-tradeoff/tests/testresources/datasets/constraints_fact_v1.0/xsum/collect.json")
# assert PATH_CONSTRAINS_XSUM.exists()

# with PATH_CONSTRAINS_XSUM.open() as f:
#     seq_dataset = [json.loads(_line) for _line in f.readlines()]
# # end with

# logger.info(f'{len(seq_dataset)} records')

# # double check: all xsum
# for _record in seq_dataset:
#     assert _record['dataset_name'] == 'xsum'
# # end for

In [ ]:
# case CNN constraints dataset
import json

PATH_CONSTRAINS_CNN = Path("/workdir/kmitsuzawa/Project/neurips-2025/ConstraintsFact-Dreyer-2023/abstractive-factual-tradeoff/tests/testresources/datasets/constraints_fact_v1.0/cnn_dailymail/collect.json")
assert PATH_CONSTRAINS_CNN.exists()

with PATH_CONSTRAINS_CNN.open() as f:
    seq_dataset = [json.loads(_line) for _line in f.readlines()]
# end with

logger.info(f'{len(seq_dataset)} records')

# double check: all xsum
for _record in seq_dataset:
    assert _record['dataset_name'] == 'cnn_dailymail'
# end for

[I 250709 10:05:20 2043887377:11] 2400 records


In [ ]:
def get_source_and_summary(record_obj: ty.Dict) -> ty.Tuple[str, str]:
    # return record_obj['document_original'], record_obj['summary_raw']
    return record_obj['document_full'], record_obj['summary_raw']
# end def

target_document_index = [1, 10, 100, 200]

import pprint

seq_stack = []

dict_commnad2ep = dict(
    lambda4 = 'log_exp(2,4.804488)',  # lambda4
    lambda2 = 'log_exp(2,2.402244)',  # lambda2
    lambda1 = 'log_exp(2,1.201122)',  # lambda1
    none = 'none()',
    linear = 'linear()',
)
dict_commnad2ep['1/lambda2'] = 'log_exp(2,0.416277447)'  # 1/lambda2, log_exp(2, 1 / (1.20112 * 2))
dict_commnad2ep['1/lambda1'] = 'log_exp(2,0.832556281)'  # 1/lambda1, log_exp(2, 1 / 1.20112)


# for _idx in target_document_index:
#     _record = seq_dataset[_idx]

#     _document_id: str = _record['document_id']
#     command_abstractiveness_constraint: str = _record['abstractiveness_constraint']

#     _document_original, _summary_raw = get_source_and_summary(_record)
#     extractive_penalty_fct = dict_commnad2ep[command_abstractiveness_constraint]

#     seq_summary = bart_sample(
#         bart=bart_model,
#         batch=[_document_original],
#         extractive_penalty_fct=extractive_penalty_fct
#     )

#     _res_obj = dict(
#         document_id=_document_id,
#         document_original=_document_original,
#         summary_raw=_summary_raw,
#         summary_gen=seq_summary[0],
#         extractive_penalty_fct=extractive_penalty_fct,
#         command_abstractiveness_constraint=command_abstractiveness_constraint
#     )
#     seq_stack.append(_res_obj)
# # end for

In [ ]:
def get_extractive_penalty_fct(penalty_command: str) -> str:
    dict_commnad2ep = dict(
        lambda4 = 'log_exp(2,4.804488)',  # lambda4
        lambda2 = 'log_exp(2,2.402244)',  # lambda2
        lambda1 = 'log_exp(2,1.201122)',  # lambda1
        none = 'none()',
        linear = 'linear()',
    )
    dict_commnad2ep['1/lambda2'] = 'log_exp(2,0.416277447)'  # 1/lambda2, log_exp(2, 1 / (1.20112 * 2))
    dict_commnad2ep['1/lambda1'] = 'log_exp(2,0.832556281)'  # 1/lambda1, log_exp(2, 1 / 1.20112)

    assert penalty_command in dict_commnad2ep

    return dict_commnad2ep[penalty_command]


def init_model(testresource: Path) -> BARTHubInterface:
    path_model = testresource / 'models/bart.large.cnn'
    assert path_model.exists(), f'no model found at {path_model}'

    bart_model = load_model(path_model, path_model / 'model.pt')

    return bart_model

def test_stochastic_sampling_sample_same(testresource: Path):
    bart_model = init_model(testresource)

    if torch.cuda.is_available():
        device_obj = torch.device('cuda:0')
    else:
        device_obj = torch.device('cpu')
    # end if

    bart_model = bart_model.to(device_obj)    

    document_input = 'He\'s a blue chip college basketball recruit. She\'s a high school freshman with Down syndrome. At first glance Trey Moses and Ellie Meredith couldn\'t be more different. But all that changed Thursday when Trey asked Ellie to be his prom date. Trey -- a star on Eastern High School\'s basketball team in Louisville, Kentucky, who\'s headed to play college ball next year at Ball State -- was originally going to take his girlfriend to Eastern\'s prom. So why is he taking Ellie instead? "She\'s great... she listens and she\'s easy to talk to" he said. Trey made the prom-posal (yes, that\'s what they are calling invites to prom these days) in the gym during Ellie\'s P.E. class. Trina Helson, a teacher at Eastern, alerted the school\'s newspaper staff to the prom-posal and posted photos of Trey and Ellie on Twitter that have gone viral. She wasn\'t surpristed by Trey\'s actions. "That\'s the kind of person Trey is," she said. To help make sure she said yes, Trey entered the gym armed with flowers and a poster that read "Let\'s Party Like it\'s 1989," a reference to the latest album by Taylor Swift, Ellie\'s favorite singer. Trey also got the OK from Ellie\'s parents the night before via text. They were thrilled. "You just feel numb to those moments raising a special needs child,"  said Darla Meredith, Ellie\'s mom. "You first feel the need to protect and then to overprotect." Darla Meredith said Ellie has struggled with friendships since elementary school, but a special program at Eastern called Best Buddies had made things easier for her. She said Best Buddies cultivates friendships between students with and without developmental disabilities and prevents students like Ellie from feeling isolated and left out of social functions. "I guess around middle school is when kids started to care about what others thought," she said, but "this school, this year has been a relief." Trey\'s future coach at Ball State, James Whitford, said he felt great about the prom-posal, noting that Trey, whom he\'s known for a long time, often works with other kids . Trey\'s mother, Shelly Moses, was also proud of her son. "It\'s exciting to bring awareness to a good cause," she said. "Trey has worked pretty hard, and he\'s a good son." Both Trey and Ellie have a lot of planning to do. Trey is looking to take up special education as a college major, in addition to playing basketball in the fall. As for Ellie, she can\'t stop thinking about prom. "Ellie can\'t wait to go dress shopping" her mother said. "Because I\'ve only told about a million people!" Ellie interjected.'
    extractive_penalty_fct = get_extractive_penalty_fct('none')

    random_seed = 42
    seq_summary_stochastic_1st = bart_sample_stochastic(
        bart=bart_model,
        temperature=1.0,
        batch=[document_input],
        random_seed_value=random_seed,
        extractive_penalty_fct=extractive_penalty_fct)
    
    seq_summary_stochastic_2nd = bart_sample_stochastic(
        bart=bart_model,
        temperature=1.0,
        batch=[document_input],
        random_seed_value=random_seed,
        extractive_penalty_fct=extractive_penalty_fct)
    
    assert seq_summary_stochastic_1st[0] == seq_summary_stochastic_2nd[0]


def test_stochastic_sampling_sample_variant(testresource: Path):
    bart_model = init_model(testresource)

    if torch.cuda.is_available():
        device_obj = torch.device('cuda:0')
    else:
        device_obj = torch.device('cpu')
    # end if

    bart_model = bart_model.to(device_obj)    

    document_input = 'He\'s a blue chip college basketball recruit. She\'s a high school freshman with Down syndrome. At first glance Trey Moses and Ellie Meredith couldn\'t be more different. But all that changed Thursday when Trey asked Ellie to be his prom date. Trey -- a star on Eastern High School\'s basketball team in Louisville, Kentucky, who\'s headed to play college ball next year at Ball State -- was originally going to take his girlfriend to Eastern\'s prom. So why is he taking Ellie instead? "She\'s great... she listens and she\'s easy to talk to" he said. Trey made the prom-posal (yes, that\'s what they are calling invites to prom these days) in the gym during Ellie\'s P.E. class. Trina Helson, a teacher at Eastern, alerted the school\'s newspaper staff to the prom-posal and posted photos of Trey and Ellie on Twitter that have gone viral. She wasn\'t surpristed by Trey\'s actions. "That\'s the kind of person Trey is," she said. To help make sure she said yes, Trey entered the gym armed with flowers and a poster that read "Let\'s Party Like it\'s 1989," a reference to the latest album by Taylor Swift, Ellie\'s favorite singer. Trey also got the OK from Ellie\'s parents the night before via text. They were thrilled. "You just feel numb to those moments raising a special needs child,"  said Darla Meredith, Ellie\'s mom. "You first feel the need to protect and then to overprotect." Darla Meredith said Ellie has struggled with friendships since elementary school, but a special program at Eastern called Best Buddies had made things easier for her. She said Best Buddies cultivates friendships between students with and without developmental disabilities and prevents students like Ellie from feeling isolated and left out of social functions. "I guess around middle school is when kids started to care about what others thought," she said, but "this school, this year has been a relief." Trey\'s future coach at Ball State, James Whitford, said he felt great about the prom-posal, noting that Trey, whom he\'s known for a long time, often works with other kids . Trey\'s mother, Shelly Moses, was also proud of her son. "It\'s exciting to bring awareness to a good cause," she said. "Trey has worked pretty hard, and he\'s a good son." Both Trey and Ellie have a lot of planning to do. Trey is looking to take up special education as a college major, in addition to playing basketball in the fall. As for Ellie, she can\'t stop thinking about prom. "Ellie can\'t wait to go dress shopping" her mother said. "Because I\'ve only told about a million people!" Ellie interjected.'
    extractive_penalty_fct = get_extractive_penalty_fct('none')

    random_seed = 42
    seq_summary_stochastic_1st = bart_sample_stochastic(
        bart=bart_model,
        temperature=1.0,
        batch=[document_input],
        random_seed_value=random_seed,
        extractive_penalty_fct=extractive_penalty_fct)
    
    random_seed = 1
    seq_summary_stochastic_2nd = bart_sample_stochastic(
        bart=bart_model,
        temperature=1.0,
        batch=[document_input],
        random_seed_value=random_seed,
        extractive_penalty_fct=extractive_penalty_fct)
    
    assert seq_summary_stochastic_1st[0] != seq_summary_stochastic_2nd[0]

# --------------------------

path_resources = Path('/workdir/kmitsuzawa/Project/neurips-2025/ConstraintsFact-Dreyer-2023/abstractive-factual-tradeoff/tests/testresources')

test_stochastic_sampling_sample_same(path_resources)
test_stochastic_sampling_sample_variant(path_resources)

[I 250709 10:24:05 2610531437:10] Loading model /workdir/kmitsuzawa/Project/neurips-2025/ConstraintsFact-Dreyer-2023/abstractive-factual-tradeoff/tests/testresources/models/bart.large.cnn/model.pt
[I 250709 10:24:25 2610531437:17] Loading done.
[I 250709 10:24:29 2610531437:10] Loading model /workdir/kmitsuzawa/Project/neurips-2025/ConstraintsFact-Dreyer-2023/abstractive-factual-tradeoff/tests/testresources/models/bart.large.cnn/model.pt
[I 250709 10:24:47 2610531437:17] Loading done.


In [ ]:
_idx = 10

_record = seq_dataset[_idx]

_document_id: str = _record['document_id']
command_abstractiveness_constraint: str = _record['abstractiveness_constraint']

_document_original, _summary_raw = get_source_and_summary(_record)
extractive_penalty_fct = dict_commnad2ep[command_abstractiveness_constraint]

seq_summary_beam = bart_sample(
    bart=bart_model,
    batch=[_document_original],
    extractive_penalty_fct=extractive_penalty_fct
)


seq_summary_stochastic = bart_sample_stochastic(
    bart=bart_model,
    temperature=1.0,
    batch=[_document_original],
    extractive_penalty_fct=extractive_penalty_fct,
    random_seed_value=42
)


_res_obj = dict(
    document_id=_document_id,
    document_original=_document_original,
    summary_raw=_summary_raw,
    summary_gen=seq_summary_beam[0],
    summary_gen_stochastic=seq_summary_stochastic[0],
    extractive_penalty_fct=extractive_penalty_fct,
    command_abstractiveness_constraint=command_abstractiveness_constraint
)


_res_obj


/home/kmitsuzawa/.local/miniconda3/envs/p39-Dreyer-2023/lib/python3.9/site-packages/torch/random.py:99: UserWarning: CUDA reports that you have 2 available devices, and you have used fork_rng without explicitly specifying which devices are being used. For safety, we initialize *every* CUDA device by default, which can be quite slow if you have a lot of GPUs.  If you know that you are only making use of a few CUDA devices, set the environment variable CUDA_VISIBLE_DEVICES or the 'devices' keyword argument of fork_rng with the set of devices you are actually using.  For example, if you are using CPU only, set CUDA_VISIBLE_DEVICES= or devices=[]; if you are using GPU 0 only, set CUDA_VISIBLE_DEVICES=0 or devices=[0].  To initialize all devices and suppress this warning, set the 'devices' keyword argument to `range(torch.cuda.device_count())`.
  warnings.warn(


{'document_id': '10',
 'document_original': 'He\'s a blue chip college basketball recruit. She\'s a high school freshman with Down syndrome. At first glance Trey Moses and Ellie Meredith couldn\'t be more different. But all that changed Thursday when Trey asked Ellie to be his prom date. Trey -- a star on Eastern High School\'s basketball team in Louisville, Kentucky, who\'s headed to play college ball next year at Ball State -- was originally going to take his girlfriend to Eastern\'s prom. So why is he taking Ellie instead? "She\'s great... she listens and she\'s easy to talk to" he said. Trey made the prom-posal (yes, that\'s what they are calling invites to prom these days) in the gym during Ellie\'s P.E. class. Trina Helson, a teacher at Eastern, alerted the school\'s newspaper staff to the prom-posal and posted photos of Trey and Ellie on Twitter that have gone viral. She wasn\'t surpristed by Trey\'s actions. "That\'s the kind of person Trey is," she said. To help make sure she 

In [ ]:
# summary_stochastic_42 = bart_sample_stochastic(
#     bart=bart_model,
#     temperature=1.0,
#     batch=[_document_original],
#     extractive_penalty_fct=extractive_penalty_fct,
#     random_seed_value=42
# )

# summary_stochastic_24 = bart_sample_stochastic(
#     bart=bart_model,
#     temperature=1.0,
#     batch=[_document_original],
#     extractive_penalty_fct=extractive_penalty_fct,
#     random_seed_value=24
# )

# summary_stochastic_42, summary_stochastic_24

(['Trey Moses is a star basketball player at Eastern High Party in Kentucky. Ellis Meredith, who has Down syndrome, entered the gym with flowers for the prom director. A teacher at the school witnessed the proposal and alerted the NICU. The photos of Trey and Ellie at the ceremony have gone viraldaughter Osiris, a special ed teacher at Eastern, tweeted about it.'],
 [" Excellence required at high school; basketball player decides to take Down syndrome girl to prom. Teacher posts photos of couple on�� on Lollapalooza;itement goes viral. Trey's now taking Ellie, an admirer of Down syndrome kids, as his date. Ellie's mother says empowering special education students janitionally helps eased Ellie's social struggles."])

In [ ]:
# document_input = 'He\'s a blue chip college basketball recruit. She\'s a high school freshman with Down syndrome. At first glance Trey Moses and Ellie Meredith couldn\'t be more different. But all that changed Thursday when Trey asked Ellie to be his prom date. Trey -- a star on Eastern High School\'s basketball team in Louisville, Kentucky, who\'s headed to play college ball next year at Ball State -- was originally going to take his girlfriend to Eastern\'s prom. So why is he taking Ellie instead? "She\'s great... she listens and she\'s easy to talk to" he said. Trey made the prom-posal (yes, that\'s what they are calling invites to prom these days) in the gym during Ellie\'s P.E. class. Trina Helson, a teacher at Eastern, alerted the school\'s newspaper staff to the prom-posal and posted photos of Trey and Ellie on Twitter that have gone viral. She wasn\'t surpristed by Trey\'s actions. "That\'s the kind of person Trey is," she said. To help make sure she said yes, Trey entered the gym armed with flowers and a poster that read "Let\'s Party Like it\'s 1989," a reference to the latest album by Taylor Swift, Ellie\'s favorite singer. Trey also got the OK from Ellie\'s parents the night before via text. They were thrilled. "You just feel numb to those moments raising a special needs child,"  said Darla Meredith, Ellie\'s mom. "You first feel the need to protect and then to overprotect." Darla Meredith said Ellie has struggled with friendships since elementary school, but a special program at Eastern called Best Buddies had made things easier for her. She said Best Buddies cultivates friendships between students with and without developmental disabilities and prevents students like Ellie from feeling isolated and left out of social functions. "I guess around middle school is when kids started to care about what others thought," she said, but "this school, this year has been a relief." Trey\'s future coach at Ball State, James Whitford, said he felt great about the prom-posal, noting that Trey, whom he\'s known for a long time, often works with other kids . Trey\'s mother, Shelly Moses, was also proud of her son. "It\'s exciting to bring awareness to a good cause," she said. "Trey has worked pretty hard, and he\'s a good son." Both Trey and Ellie have a lot of planning to do. Trey is looking to take up special education as a college major, in addition to playing basketball in the fall. As for Ellie, she can\'t stop thinking about prom. "Ellie can\'t wait to go dress shopping" her mother said. "Because I\'ve only told about a million people!" Ellie interjected.'
# extractive_penalty_fct = get_extractive_penalty_fct('none')

# # tensor_source_tokens = bart_model.encode([document_input])
# # tensor_source_tokens = torch.tensor([tensor_source_tokens]).to(bart_model.device)

# beam: int = 4
# lenpen: float = 2.0  # length penalty
# min_len: int = 55
# max_len_a: int = 0
# max_len_b: int = 140
# no_repeat_ngram_size: int = 3

# random_seed_value = 42

# dict_parameters = dict(
#     beam=beam,
#     lenpen=lenpen,
#     sampling=True,
#     temperature=1.0,
#     min_len=min_len, 
#     max_len_a=max_len_a, 
#     max_len_b=max_len_b,
#     no_repeat_ngram_size=no_repeat_ngram_size,
#     extractive_penalty_fct=extractive_penalty_fct)

# with (torch.random.fork_rng(), torch.no_grad()):
#     torch.manual_seed(random_seed_value)
#     torch.cuda.manual_seed_all(random_seed_value)  # if you are using multi-GPU.        

#     res = bart_model.generate(tensor_source_tokens)



NameError: name 'get_extractive_penalty_fct' is not defined